In [1]:
import pandas as pd
titanic = pd.read_csv("input/titanic_train.csv")
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#使用describe 对所有的样本进行 计算样本的count，通过count发现 数据 中 部分特征有缺失值，（Age下面有缺失值，数据有缺失）
#数据缺失 导致行数 对不上，数据缺失会影响构造矩阵，影响使用算法
#缺失值的解决办法：1、抛弃无效数据 2、在数据很重要的情况下，进行数据的填充，使用均值进行填充，


#对于数值数据，结果的索引将包括计数，平均值，标准差，最小值，最大值以及较低的百分位数和50。
#默认情况下，较低的百分位数为25，较高的百分位数为75.50百分位数与中位数相同。

# print(titanic.describe()) 

In [3]:
#通过describe（）方法发现Age特征中的数据有缺失，但是当前特征比较重要
#遂对Age特征进行数据填充(填充通过该特征的均值进行填充)
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
print(titanic.describe())
#note：当前显示的量都是数值的量，没有字符的量比如性别：male female
#note:给机器学习算法字符是不行的，不能转换为矩阵，字符不能转换为矩阵
#note: 对字符的量要转换为数值的量

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [4]:
# 由于机器学习算法 转换为矩阵需要数值，而不能识别字符，则需要对部分特征 字符进行
# 转换，转换为数值，然后进行numpy 矩阵计算
#对数据进行去重,获取特征值，Sex的特征下的类别
print(titanic["Sex"].unique())

#替换性别的两个类别，用0代表male,1代表female
#字符到数值的转换
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1


['male' 'female']


In [5]:
#输出Emarked特征下的类别 
#输出后发现Embarked 下面有空值，但是由于是字符类型，使用均值不太合适，
#此时字符的缺失值，一般用最多的类别去填充，S最多，则填充S
print(titanic["Embarked"].unique())

titanic["Embarked"] = titanic["Embarked"].fillna('S')
#数值映射 
titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

# print(titanic.describe())
#当前代码运行第二遍 会报错，因为 数据已经转换完毕，再次转换就会出现问题，重新运行所有代码即可


['S' 'C' 'Q' nan]


In [17]:
#使用线性回归预测
from sklearn.linear_model import LinearRegression
import datetime
from sklearn.cross_validation import KFold

#机器学习项目，一份数据来了之后，首先分成两份，测试集和训练集，
#交叉验证，将训练集分成10份，将9份作为训练集，一份作为验证集，
#cross_validation 交叉验证，123 训练 4测试，134训练 2测试


#自己去实现线性回归算法比较麻烦，一般情况下还是使用 第三方库进行分析

#第一步：提取特征，
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

#初始化模型
alg = LinearRegression()

#做交叉验证 自选做几倍的交叉验证
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)
predictions = []

# kf对象里有很多的训练集 和 测试集 ，我们需要对每一份 都要
# 拿到他的训练集和测试集
for train, test in kf:
    
    #获取训练集
    train_predictors = (titanic[predictors].iloc[train,:])
    
    #获取标签 特征标签 Survived
    train_target = titanic["Survived"].iloc[train]
#     print(train_target)
    # 传入 训练集和标签 进行训练模型
    alg.fit(train_predictors, train_target)
    #线性回归模型 拟合完毕 获取的数据之后，就可以进行预测
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    #将每一个预测集 添加进数组
    predictions.append(test_predictions)

# print(predictions)


In [18]:
import numpy as np

predictions = np.concatenate(predictions, axis=0)

#将获取的值映射到0和1，大于0.5 = 1 小于0.5 = 0
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0

# print(predictions)
#将预测值和真实值进行比较 预测值 是否和 样本值相近
# print(predictions[predictions == titanic["Survived"]])
#判断正确个数
print(sum(predictions[predictions == titanic["Survived"]]))
#获取准确率
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print(accuracy)

[0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1.

In [10]:
#想使用概率值 就可以使用逻辑回归来做
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg,titanic[predictors], titanic["Survived"], cv=3)
print(scores.mean())

0.7878787878787877


In [19]:
#以上都是在训练集上进行衡量，应该在测试集上衡量
#当模型参数调好之后，使用测试集进行测试
#处理完测试集数据后，用一开始的模型 去 验证测试集
titanic_test = pd.read_csv("input/test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic["Age"].median())
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2
#处理完测试集数据后，用一开始的模型 去 验证测试集

In [22]:



from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
#n_estimators：构造多少颗树  min_samples_split：最小的切分点，样本切分 min_samples_leaf：叶子结点最少的个数
#min_samples_split min_samples_leaf 控制树的高度，防止过拟合
#随机森林可能会建的很大，会发生过拟合的现象，不要为了单独的噪音点切分 ，树不要建立太高
 
#随机森林的调参：建立的树的多少？树大一些效果会好一点，min_samples_split min_samples_leaf 节点分割不要太多
alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, min_samples_leaf=1)
#做交叉验证
kf = cross_validation.KFold(titanic.shape[0], n_folds=3, random_state=1)
#交叉验证cv 赋值
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=kf)

# Take the mean of the scores (because we have one for each fold)
print(scores.mean())
#获取到的结果竟然和其他分类器差不多

0.7856341189674523


In [21]:
#对随机森林 进行调参
#随机森林可能会见的很大，会发生过拟合的现象，不要为了单独的噪音点切分
alg = RandomForestClassifier(random_state=1, n_estimators=50, min_samples_split=4, min_samples_leaf=2)
#做交叉验证
kf = cross_validation.KFold(titanic.shape[0], n_folds=3, random_state=1)
#交叉验证cv 赋值
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=kf)

# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.8159371492704826


In [ ]:
#模型调参后，回到数据层面
titanic